<a href="https://colab.research.google.com/github/jyeongvv/DL_study/blob/main/jyeongvv/0331_DNN%2C_CNN%2C_ResNet_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DNN

## Transforms에서 자주 쓰이는 기능

* ToTensor : 이미지를 파이토치 텐서로 변환
* Resize : 이미지 크기 조정
* Normalize : 주어진 평균과 표준편차를 이용하여 정규화
* RandomHorizontalFlip : 무작위로 이미지의 오른쪽과 왼쪽을 뒤집는 기능
* RandomCrop : 이미지를 무작위로 자르는 기능

데이터셋
* torchvision.datasets로 생성된 객체 -> 파이토치 내부 클래스 torch.utils.data.Dataset을 상속
* 클래스에 넣어준 후 데이터 로딩 -> 사용 가능
* 배치 크기(batch size)를 지정
* iter() : 반복되는 객체를 반복문 안에 이용할 수 있도록 만들어주는 함수
* next() : 배치 1개를 (아직 반복되지 않은 하나의 원소를) 불러와주는 함수
* utils.make_grid() -> 여러 이미지를 모아서 하나의 이미지로
* np.transpose -> 차원의 순서를 교체
* 데이터셋에서는 이름 대신에 숫자 번호로 레이블이 주어짐 -> 딕셔너리화
* 이미지의 데이터는 가로(픽셀수), 세로(픽셀수), 색상 3차원 행렬로 표현

## 인공신경망

* USE_CUDA
* 배치사이즈와 에포크 설정해주기

## 데이터셋 불러오기

* ToTensor를 통해 이미지를 파이토치 Tensor화

## 이미지 분류
`image claasification`
* 이미지 한장을 받아 이 이미지가 어느 클래스에 속해있는지 알려줌
* 단순하지만 영향력 강한편
* 비디오 분류 모델(연속된 이미지)도 기본적인 이미지 분류방식과 크게 다르지 않음

## 이미지 분류를 위한 인공 신경망
* 입력 x와 레이블(정답) y를 받아서 학습한 다음, 새로운 X가 왔을 때 어떤 패션 아이템인지 예측하는 모델

### 인공 신경망 구성
```
 nn.Linear 클래스는 선형 결합을 수행하는 객체를 만듦
        self.fc1 = nn.Linear(784, 256) # input 784, output 256
        # 1 x 28 x 28 -> 784 픽셀값(한 점씩)을 입력 받아서 가중치를 행렬곱/편향. 값 256개를 출력
        # fc는 fully connected -> 이전 레이어의 모든 노드가 다음 레이어의 모든 노드에 연결된 층 (Dense) 
        
        self.fc2 = nn.Linear(256, 128) # input 256, output 128
        self.fc3 = nn.Linear(128, 10) # input 128, output 10
        # 출력값 10개 각각은 클래스가 될 확률을 나타내며, 10개 중 값이 가장 큰 클래스가 이 모델의 예측값의 될 것임
```


```
훈련 과정을 표현
    def forward(self, x): # x는 입력되는 텐서(input_tensor)
        # 데이터의 흐름을 정의
        # n, 색상, 세로, 가로 ->  (-1, 784) # 64, 784 (배치 사이즈 만큼)
        # 입력을 받아 view() 함수를 이용하여 랭크 1 텐서로 변환
        x = x.view(-1, 784) # 28px x 28px인 이미지 텐서를 784 한 줄짜리 1차원 텐서
        # -1 : 나머지값들. (배치 사이즈, 데이터 사이즈...)
        
        # fc1()과 fc2()를 거치게 할 것, 각각 층은 ReLU 활성화 함수를 통해서 처리
        # F.relu == nn.ReLU (거의 동일, 취향 문제) -> 가중치가 없는 연산 (저장 X)
        # nn.Linear -> 선형결합 (가중치) -> 거의 강제. (init(생성자)에 선언하는 것이 권장)
        # x = self.fc1(x)
        # x = F.relu(x)
        x = F.relu(self.fc1(x)) # 입력층 - 은닉층1
        x = F.relu(self.fc2(x)) # 입력층 - 은닉층2

        # 마지막으로 fc3() 함수까지 거쳐 나온 값 10개
        x = self.fc3(x)
        return x
```

* to() 함수는 모델의 파라미터들을 지정한 장치의 메모리로 **보내는** 역할을 함

최적화 알고리즘
* 최적화 알고리즘
  * 파이토치 내장 모듈인 optim.SGD를 사용
  * SGD (확률적 경사하강법, stochastic gradient descent) : 모델 최적화를 확률적 경사하강법
  * 모델 내부의 정보를 넘겨주는 model.parameters() 함수와 직접 설정한 학습률(lr)을 입력

## 학습

* train : 학습할 model, 데이터를 공급해주는 train_loader, 최적화를 담당할 optimizer를 넣을 것
* `model.train()` : 학습 모드 <-> `model.eval()` : 평가모드
* enumerate(연속된 데이터) -> zip(인덱스, 값)
* data : 모델에 입력할 data (이미지 텐서), target : 해당 이미지의 분류값. 클래스.
* 모델의 가중치를 보낸 장치(DEVICE) -> 학습 데이터도 같은 장치로 보내야 함
* 배치를 처리할 때마다 기울기를 새로 계산해줘야함 -> optimizer.zero_grad()
* 학습 데이터에 대한 모델의 예측값을 output
        `output = model(data)`
* 이진 교차 엔트로피 binary cross entropy : 클래스가 2개 (분류해야하는 대상이 2개)
  * 교차 엔트로피 cross entropy : 클래스가 3개 이상
* `optimizer.step()` -> 계산한 기울기를 바탕으로 앞서 정의한 알고리즘에 맞추어 가중치를 수정

* 일반화 `generalization` : 세상에 존재하는 모든 데이터에 최적화하는 것
* 일반화 오류 `generalization error` : 학습 데이터를 기반으로 한 모델이 학습하지 않은 데이터에 얼마나 적응하는지를 수치로 나타낸 것
    * 일반화 오류는 작을수록 좋음
* 평가용 데이터셋 = 실제 세상의 모든 데이터를 대표함
* 학습(7), 검증(2), 테스트(1) 3단계로 나뉨
* **하이퍼 파라미터** : 모델 층의 종류와 크기, 배치 크기, 학습률 등 머신러닝 모델이 배우지 않고 사용자가 직접 지정해주는 값
  * 모델의 가중치를 최적화하는데 영향을 줌
  * 데이터 모으는 과정만큼 중요함

**학습**
```
# 평가 함수 evaluate
# 모델의 일반화 정도 확인 + 학습을 언제 멈춰야할지 알아냄 (조기종료)
def evaluate(model, test_loader): # 최적화 담당하는 optimizer 없어도 됌
    # epoch가 끝날 때마다 테스트셋으로 모델의 성능을 측정
    model.eval() # 평가 모드

    test_loss = 0 # 테스트셋의 오차
    correct = 0 # 예측이 맞은 갯수를 담아줄 변수

    # torch.no_grad() -> 생성 / with가 끝나면 해당 객체, 기능을 close.
    with torch.no_grad(): # 기울기 계산 끄기
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE) # DEVICE로 데이터 보내기
            output = model(data) # 테스트 데이터의 예측값

            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item() # item() 스칼라 텐서 안에 있는 숫자값을 파이썬의 숫자 자료형 형태로 뽑아주는 함수
            # 평가를 위해서 교차 엔트로피를 거칠 때 reduction을 'sum'을 지정
            # 원래 default는 mean(평균)
            # 모든 오차를 더해줌
            # ---
            # 정확도
            # 가장 큰 값을 가진 클래스가 모델의 
            # -> output.max(1, ...)[1] -> 가장 높게 나온 확률을 가진 패션 아이템의 종류 (인덱스)를 추출하는 코드
            # 예측과 정답을 비교하여서 일치할 경우 -> correct에 1을 더함 
            # output.max() -> (가장 큰 값(0), 그 값이 있는 자리의 인덱스(1))
            # print(output)
            # https://pytorch.org/docs/stable/generated/torch.Tensor.max.html
            pred = output.max(1, keepdim=True)[1] # max(차원)
            # print(pred)

            # 모델의 예측 패션 아이템과 레이블(정답)이 일치하는지는 eq (0, 1)
            # print(pred.eq(target.view_as(pred)))
            # target.view_as(pred) -> target을 pred의 모양으로 변경
            correct += pred.eq(target.view_as(pred)).sum().item() # 일치하는 것만 -> 1, 0 -> count -> item() (스칼라)
            # -> sum (모두 더하면 현 배치에서 모델을 맞춘 정답 개수 구함)
            # view_as -> 인수로 들어간 텐서의 모양으로 바꿔줌(정렬)
    # 배치별 총 오차와, 총 맞춘 개수를 합친 test_loss, correct

    # test_loss : 전체 데이터셋에 대한 오차 -> 배치별로 나눠져 있는 것의 합
    # correct : 맞힌 개수의 합
    # 총 정답 평균 -> 100을 곱해서 정확도 (accuracy) 구하기
    test_loss /= len(test_loader.dataset) # 평균오차
    test_accuracy = 100. * correct / len(test_loader.dataset) # 얼마나 맞췄는지
    return test_loss, test_accuracy
```

* 읽어보기 -> 어떻게 흘러가는지 아는게 중요(흐름)

### 과적합 / dropout
* 과적합 `overfitting`
  * 학습 성능은 좋은데, 테스트셋이나 실제 상황에서는 성능이 안나옴
  * 학습 데이터에만 치중되어 새로운 데이터에서는 성능이 잘 나오지 않는 상황
* 과소적합 `underfitting`
  * 학습을 제대로 진행하지 않은 상황
  * 학습 데이터도 제대로 학습하지 않은 경우

**바람직한 상태?** -> 과소적합과 과적합의 중간상태!
* 일반화 = 학습데이터와 그렇지 않은 데이터(실제)에서 동시에 높은 성능을 내는 것
* 분류형 -> 일반화를 지향함
* 검증 오차가 올라가는 순간을 포착하여 학습을 종료하는 것을 **조기 종료**`early stopping`라고 함


데이터 늘리기
* data augmentation 적용
* 이미지 데이터의 경우 -> 이미지의 일부분 자르기, 돌리기, 노이즈를 더하기, 색상 변경... 여러 방법 사용 가능

drop out
* 신경망의 일부를 사용하지 않는 방법 -> 모델에 직접 영향을 줌
* 학습에서 배제된 뉴런 외에 다른 뉴런들에 가중치를 분산시키고 개별 뉴런이 특징에 고정되는 현상을 방지하는 기능 (영향력이 큰 몇몇 요소들을 의도적으로 배제)

# CNN

* CNN은 컨볼루션, 폴링, 드롭아웃, 일반적인 신경망 계층의 조합으로 이루어짐

* 코드를 간결하게 하기 위해서 Fashion MNIST 데이터셋을 DataLoader를 부를 때 바로 정의
* transfroms를 이용한 전처리 → 파이토치 텐서화, 정규화 진행
   * 파이토치에서 CNN 구현 -> nn.Module 상속받기 / torchvision 패키지를 사용

* nn.Conv2d 모듈 -> 입력 x를 받는 함수
  * nn.Conv2d(입력 채널 수, 출력 채널 수)
* self.conv1, self.conv2 -> CNN 모델의 내부 변수(속성)들은 함수로 취급
* 필터(커널) 크기 = kernel_size로 지정 -> 숫자 하나만 지정하면 정사각형으로 간주
*  x = F.max_pool2d(x, n) # 풀링도 커널사이즈가 있음 (nxn)
* **활성화 함수 F.relu** 거치도록!!

* fc -> 선형결합을 해서 한줄로 펴준 다음에 우리가 분류하기를 원하는 클래스 값만큼 특징을 다시 분류


* `loss = F.cross_entropy(output, target)` -> 크로스 엔트로피 오차 함수로 오차값 계산
* 예측값과 정답값 사이의 교차 엔트로피 -> 손실(에러) 계산
*  `optimizer.step()` -> 모델의 학습 패러미터 갱신 / 가중치 (conv도 갱신, linear 갱신)
* `loss.backward()` -> 오차역전파 -> 미분계수 전달
* ` optimizer.zero_grad()` -> 배치 돌 때마다 기울기값(최적화) 리셋


```
- DNN으로 구현했던 FashionMNIST를 CNN으로 개선
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))
```

# ResNet



